Group Members 11: Jerry Yan, Annie Tsoromocos, Siddh Patel, Miles Da Silva
Github links:
https://github.com/owid/covid-19-data/tree/master/public/data/ , 

https://github.com/CSSEGISandData/COVID-19 , 

https://www150.statcan.gc.ca/n1/pub/13-26-0003/132600032020001-eng.htm 


Title: 

Research question: Can a country's Gross Domestic Product (GDP) and life expectancy be used to predict how many total Sars-CoV-2 they have had?

Introduction:

Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
Clearly state the question you will try to answer with your project
Identify and describe the dataset that will be used to answer the question
Preliminary exploratory data analysis:
Demonstrate that the dataset can be read from the web into R 
Clean and wrangle your data into a tidy format
Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

Methods:


Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
Describe at least one way that you will visualize the results
Expected outcomes and significance:


What do you expect to find?
What impact could such findings have?
What future questions could this lead to?



In [6]:
library(tidyverse)
library(tidymodels)
library(gridExtra)

In [7]:
covid_data <- read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv")

Parsed with column specification:
cols(
  .default = col_double(),
  iso_code = col_character(),
  continent = col_character(),
  location = col_character(),
  date = col_date(format = ""),
  icu_patients = col_logical(),
  icu_patients_per_million = col_logical(),
  hosp_patients = col_logical(),
  hosp_patients_per_million = col_logical(),
  weekly_icu_admissions = col_logical(),
  weekly_icu_admissions_per_million = col_logical(),
  weekly_hosp_admissions = col_logical(),
  weekly_hosp_admissions_per_million = col_logical(),
  new_tests = col_logical(),
  total_tests = col_logical(),
  total_tests_per_thousand = col_logical(),
  new_tests_per_thousand = col_logical(),
  new_tests_smoothed = col_logical(),
  new_tests_smoothed_per_thousand = col_logical(),
  positive_rate = col_logical(),
  tests_per_case = col_logical()
  # ... with 10 more columns
)

See spec(...) for full column specifications.

Warning message:
“1168414 parsing failures.
 row                        col           

In [8]:
# FILTER RELEVANT COLS

filtered_covid_data <- filter(covid_data, continent == "North America", date == "2022-02-28")
relevant_covid_data <- filtered_covid_data %>% 
            select(location, date, total_cases, total_cases_per_million, total_deaths, total_deaths_per_million, life_expectancy, gdp_per_capita, human_development_index)
# relevant_covid_data

In [9]:
# CHANGING NA VALUES

mty_list_gdp = c(17226, NA, NA, NA, NA, NA, NA, 24200, 34246, NA, NA, NA, 6630, 19701, NA, NA, NA, 54470.96, 
             NA, NA, NA, NA, NA, NA, 13523, NA, NA, NA, NA, 46200, NA, NA, 23879.93, NA)

# Possibly change 46200 value, since Saint Pierre and Miquelon's gdp is an old estimate (2006)

mty_list_hdi = c(0.865, NA, 0.908, NA, NA, NA, 0.981, 0.944, 0.945, NA, 0.888, NA , NA, 0.811, NA, NA, NA, 
                 0.839, NA, NA, NA, NA, NA, NA, 0.821, NA, NA, NA, NA, 0.788, NA, NA, 0.873, NA)

gdp = relevant_covid_data$gdp_per_capita
new_gdp = coalesce(gdp, mty_list_gdp)

hdi = relevant_covid_data$human_development_index
new_hdi = coalesce(hdi, mty_list_hdi)

tidy_covid_data = relevant_covid_data
tidy_covid_data$gdp_per_capita = new_gdp
# tidy_covid_data

tidy_covid_data$human_development_index = new_hdi
tidy_covid_data

location,date,total_cases,total_cases_per_million,total_deaths,total_deaths_per_million,life_expectancy,gdp_per_capita,human_development_index
<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Anguilla,2022-02-28,2555,168925.620,9,595.041,81.88,17226.000,0.865
Antigua and Barbuda,2022-02-28,7437,75328.174,135,1367.393,77.02,21490.943,0.778
Aruba,2022-02-28,33684,314231.074,211,1968.375,76.29,35973.781,0.908
Bahamas,2022-02-28,33135,83481.560,771,1942.486,73.92,27717.847,0.814
Barbados,2022-02-28,55165,191739.541,316,1098.336,79.19,16978.068,0.814
Belize,2022-02-28,56694,140014.571,650,1605.275,74.62,7824.362,0.716
Bermuda,2022-02-28,11561,186191.458,123,1980.932,82.59,50669.315,0.981
Bonaire Sint Eustatius and Saba,2022-02-28,7582,286708.262,31,1172.244,77.79,24200.000,0.944
British Virgin Islands,2022-02-28,6085,200013.148,62,2037.932,79.07,34246.000,0.945
